In [1]:
import sys
import os
import glob
import time
import pandas as pd
import ast

sys.path.append('..')
import constants.constants as const
import constants.file_handler_constants as fh
from constants.restaurant_constants import *

from packages.restaurant.Restaurant import *
from packages.file_handler_package.file_handler import *


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\user\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\user\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\user\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "c:\Users\user\anaconda3\Lib\site-packages

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\user\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\user\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\user\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "c:\Users\user\anaconda3\Lib\site-packages

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\user\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\user\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\user\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "c:\Users\user\anaconda3\Lib\site-packages

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\user\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\user\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\user\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "c:\Users\user\anaconda3\Lib\site-packages

AttributeError: _ARRAY_API not found

In [2]:
def map_type_tripAdvisor_to_wongnai(type: list[str], website:str) -> list[str]:
    
    if("tripadvisor.com" not in website):
        return

    res_type = []
    for cur_type in type:
        find_type = TYPE_TRIPADVISOR_TO_WONGNAI_MAPPER.get(cur_type, 0)
        if(find_type and find_type not in res_type):
            res_type.append(find_type)
    
    return res_type.copy()

In [3]:
# this module is use for merge dataframe of all page in selected province from 'mulProcess_restaurant_scraping_proxy'

# ** select a province by index of 'ALL_RESTAURANTS_TRIPADVISOR_DATA' (manually)
Idx_of_region = 0
cur_region_data = ALL_RESTAURANTS_TRIPADVISOR_DATA[Idx_of_region]

#
cur_province_en = cur_region_data[0]

all_res_scraping_province = glob.glob(os.path.join(fh.STORE_RESTAURANT_SCRAPING, "res_restaurant_scraping", 'res_restaurant_%s' % (cur_province_en), '*.csv'))

res_merge_df = pd.DataFrame()
for cur_page_path in all_res_scraping_province:
    cur_page_df = pd.read_csv(cur_page_path)
    res_merge_df = pd.concat([res_merge_df, cur_page_df])

# remove duplicate attraction 
res_merge_df.drop_duplicates(subset=['name'], inplace=True)
# remove row with missing latitude
# Drop rows with 'latitude' or 'longitude' equal to 0
res_merge_df = res_merge_df[(res_merge_df['latitude'] != 0) & (res_merge_df['longitude'] != 0)]
res_merge_df = res_merge_df[(res_merge_df['latitude'].notnull()) & (res_merge_df['longitude'].notnull())]

# map type from tripadvisor to wongnai
res_merge_df['type'] = res_merge_df.apply(lambda x: map_type_tripAdvisor_to_wongnai(
    type = ast.literal_eval(x['type']),
    website = x['website']
), axis=1)

# set new index
res_merge_df.set_index(['name'], inplace=True)

# create directory to store result of merging restaurant by a province
createDirectory(fh.STORE_MERGE_RESTAURANT_SCRAPING, 'res_merge_restaurant')

# save result dataframe to .csv 
# for example: 'res_restaurant_Mukdahan_page_1_44.csv'
res_file_name = 'res_merge_restaurant_%s.csv' % (cur_province_en)
res_path = os.path.join(fh.STORE_MERGE_RESTAURANT_SCRAPING, 'res_merge_restaurant', res_file_name) 
res_merge_df.to_csv(res_path, encoding="utf-8")


Directory res_merge_restaurant created successfully
